## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.58,72,3,14.32,clear sky
1,1,Carnarvon,AU,-24.8667,113.6333,80.67,54,98,10.36,overcast clouds
2,2,Petropavlovka,RU,50.6114,105.3196,43.09,44,100,4.05,overcast clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,60.10,82,0,14.00,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,55.47,54,19,14.16,few clouds


In [43]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [44]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.58,72,3,14.32,clear sky
1,1,Carnarvon,AU,-24.8667,113.6333,80.67,54,98,10.36,overcast clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,60.10,82,0,14.00,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,55.47,54,19,14.16,few clouds
5,5,Kutum,SD,14.2000,24.6667,71.31,10,77,5.61,broken clouds
...,...,...,...,...,...,...,...,...,...,...
453,453,Alofi,NU,-19.0595,-169.9187,78.69,94,17,6.91,few clouds
454,454,Yeppoon,AU,-23.1333,150.7333,78.58,60,9,12.55,clear sky
456,456,Crotone,IT,39.0851,17.1178,61.20,56,100,3.42,overcast clouds
457,457,Lardos,GR,36.0942,28.0157,57.02,53,100,2.68,overcast clouds


In [45]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             313
City                   313
Country                309
Lat                    313
Lng                    313
Max Temp               313
Humidity               313
Cloudiness             313
Wind Speed             313
Current Description    313
dtype: int64

In [46]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

Unnamed: 0             309
City                   309
Country                309
Lat                    309
Lng                    309
Max Temp               309
Humidity               309
Cloudiness             309
Wind Speed             309
Current Description    309
dtype: int64

In [50]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.58,clear sky,-23.1203,-134.9692,
1,Carnarvon,AU,80.67,overcast clouds,-24.8667,113.6333,
3,Mar Del Plata,AR,60.10,clear sky,-38.0023,-57.5575,
4,Mataura,NZ,55.47,few clouds,-46.1927,168.8643,
5,Kutum,SD,71.31,broken clouds,14.2000,24.6667,
7,Bengkulu,ID,78.44,overcast clouds,-3.8004,102.2655,
8,Busselton,AU,66.13,overcast clouds,-33.6500,115.3333,
9,Shache,CN,61.72,clear sky,38.4167,77.2406,
10,Asau,RO,52.95,overcast clouds,46.4333,26.4000,
11,Niamey,NE,82.69,few clouds,13.5137,2.1098,


In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.58,clear sky,-23.1203,-134.9692,People ThankYou
1,Carnarvon,AU,80.67,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon
3,Mar Del Plata,AR,60.10,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,Mataura,NZ,55.47,few clouds,-46.1927,168.8643,Ellie's Villa
5,Kutum,SD,71.31,broken clouds,14.2000,24.6667,Dibbo areaمنطقة دبو
7,Bengkulu,ID,78.44,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
8,Busselton,AU,66.13,overcast clouds,-33.6500,115.3333,Observatory Guest House
9,Shache,CN,61.72,clear sky,38.4167,77.2406,Delong Hotel
10,Asau,RO,52.95,overcast clouds,46.4333,26.4000,Casa Mika Ciobanus
11,Niamey,NE,82.69,few clouds,13.5137,2.1098,Soluxe Hotel Niamey


In [52]:
hotel_df.count()

City                   309
Country                309
Max Temp               309
Current Description    309
Lat                    309
Lng                    309
Hotel Name             309
dtype: int64

In [53]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   309
Country                309
Max Temp               309
Current Description    309
Lat                    309
Lng                    309
Hotel Name             309
dtype: int64

In [54]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [70]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>and<dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [74]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))